In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms

# Define the NEAT network class
class NEATNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NEATNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Define the training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data.view(data.size(0), -1))
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Define the evaluation function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.view(data.size(0), -1))
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))

# Set the device to CUDA if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the NEAT parameters
input_size = 28 * 28  # MNIST image size
output_size = 10  # Number of classes (0-9)

# Define the NEAT population size
population_size = 100

# Create NEAT population
population = []
for _ in range(population_size):
    model = NEATNetwork(input_size, output_size).to(device)
    population.append(model)

batch_size = 4
test_batch_size = 4
learning_rate = 0.001
num_epochs = 10 

# Load MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
                   batch_size=test_batch_size, shuffle=True)


optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
test(model, device, test_loader)



100%|██████████| 9912422/9912422 [00:03<00:00, 2968132.00it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 101739.56it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1099259.89it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4704995.99it/s]


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313139
Train Epoch: 1 [400/60000 (1%)]	Loss: 0.680033
Train Epoch: 1 [800/60000 (1%)]	Loss: 0.606309
Train Epoch: 1 [1200/60000 (2%)]	Loss: 1.357435
Train Epoch: 1 [1600/60000 (3%)]	Loss: 1.393335
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.030664
Train Epoch: 1 [2400/60000 (4%)]	Loss: 0.160834
Train Epoch: 1 [2800/60000 (5%)]	Loss: 0.041780
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.484438
Train Epoch: 1 [3600/60000 (6%)]	Loss: 0.063582
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.162986
Train Epoch: 1 [4400/60000 (7%)]	Loss: 0.426274
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.209715
Train Epoch: 1 [5200/60000 (9%)]	Loss: 0.086154
Train Epoch: 1 [5600/60000 (9%)]	Loss: 0.711706
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.054768
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.052949
Train Epoch: 1 [6800/60000 (11%)]	Loss: 0.189488
Train Epoch: 1 [7200/60000 (12%)]	Loss: 0.089781
Train Epoch: 1 [76

------------------